<a href="https://colab.research.google.com/github/Ferrari-coder/novelty-detection-roadmapr/blob/main/roadmapr_novelty_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from scipy.signal import butter, filtfilt
import logging
from typing import List, Tuple, Dict


class RoadDataProcessor:
    def __init__(self,
                 epoch_duration: float = 2.0,
                 overlap: float = 0.5,
                 sampling_rate: int = 100):
        """
        Initialize the road data processor.

        Args:
            epoch_duration: Duration of each epoch in seconds
            overlap: Overlap between epochs (0.5 = 50%)
            sampling_rate: Expected sampling rate of the data in Hz
        """
        self.epoch_duration = epoch_duration
        self.overlap = overlap
        self.sampling_rate = sampling_rate
        self.setup_logging()

    def setup_logging(self):
        """Configure logging for the processor."""
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
        self.logger = logging.getLogger(__name__)

    def load_csv_files(self, directory: str) -> Dict[str, pd.DataFrame]:
        """
        Load all CSV files from specified directory.

        Args:
            directory: Path to directory containing CSV files

        Returns:
            Dictionary mapping filenames to DataFrames
        """
        data_files = {}
        directory_path = Path(directory)

        try:
            # Get all CSV files in the directory
            csv_files = list(directory_path.glob('*.csv'))
            self.logger.info(f"Found {len(csv_files)} CSV files in {directory}")

            for file_path in csv_files:
                try:
                    # Read CSV file
                    df = pd.read_csv(file_path)
                    # Verify expected columns exist
                    required_columns = ['time', 'ax', 'ay', 'az', 'wx', 'wy', 'wz']
                    if not all(col in df.columns for col in required_columns):
                        self.logger.warning(f"Missing required columns in {file_path.name}")
                        continue

                    # Store DataFrame in the dictionary with filename (without extension) as key
                    data_files[file_path.stem] = df
                    self.logger.info(f"Loaded {file_path.name}: {len(df)} rows")

                except Exception as e:
                    self.logger.error(f"Error loading {file_path.name}: {str(e)}")

        except Exception as e:
            self.logger.error(f"Error accessing directory {directory}: {str(e)}")

        return data_files

    def preprocess_data(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Apply preprocessing steps to the data.

        Args:
            df: Input DataFrame with sensor readings

        Returns:
            Preprocessed DataFrame
        """
        # Create a copy to avoid modifying original data
        processed = df.copy()

        # Calculate magnitude of acceleration and angular velocity
        processed['acc_magnitude'] = np.sqrt(
            processed['ax'] ** 2 + processed['ay'] ** 2 + processed['az'] ** 2
        )
        processed['gyro_magnitude'] = np.sqrt(
            processed['wx'] ** 2 + processed['wy'] ** 2 + processed['wz'] ** 2
        )

        # Apply low-pass filter to remove high-frequency noise
        nyquist = self.sampling_rate * 0.5
        cutoff = 10  # 10 Hz cutoff frequency
        b, a = butter(4, cutoff / nyquist, btype='low')

        for col in ['ax', 'ay', 'az', 'acc_magnitude']:
            processed[f'{col}_filtered'] = filtfilt(b, a, processed[col])

        return processed

    def create_epochs(self, df: pd.DataFrame) -> List[Dict]:
        """
        Create overlapping epochs from the data.

        Args:
            df: Preprocessed DataFrame

        Returns:
            List of dictionaries containing epoch data and metadata
        """
        samples_per_epoch = int(self.epoch_duration * self.sampling_rate)
        stride = int(samples_per_epoch * (1 - self.overlap))
        epochs = []

        for i in range(0, len(df) - samples_per_epoch + 1, stride):
            epoch_data = df.iloc[i:i + samples_per_epoch]

            epoch = {
                'start_time': epoch_data['time'].iloc[0],
                'end_time': epoch_data['time'].iloc[-1],
                'data': epoch_data,
                'index': len(epochs)
            }

            epochs.append(epoch)

        self.logger.info(f"Created {len(epochs)} epochs")
        return epochs

    def process_file(self, df: pd.DataFrame, road_id: str) -> List[Dict]:
        """
        Process a single file's data into epochs.

        Args:
            df: Input DataFrame
            road_id: Identifier for the road/file

        Returns:
            List of processed epochs with metadata
        """
        # Preprocess the data
        processed_data = self.preprocess_data(df)

        # Create epochs
        epochs = self.create_epochs(processed_data)

        # Add road identifier to each epoch
        for epoch in epochs:
            epoch['road_id'] = road_id

        return epochs

    def process_all_files(self, directory: str) -> Tuple[List[Dict], Dict[str, int]]:
        """
        Process all CSV files in directory into epochs.

        Args:
            directory: Path to directory containing CSV files

        Returns:
            Tuple of (list of all epochs, dictionary of epochs per road)
        """
        data_files = self.load_csv_files(directory)
        all_epochs = []
        epochs_per_road = {}

        for road_id, df in data_files.items():
            road_epochs = self.process_file(df, road_id)
            all_epochs.extend(road_epochs)
            epochs_per_road[road_id] = len(road_epochs)

        self.logger.info(f"Total epochs created: {len(all_epochs)}")
        return all_epochs, epochs_per_road


def main():
    # Example usage
    processor = RoadDataProcessor(
        epoch_duration=2.0,
        overlap=0.5,
        sampling_rate=100
    )

    # Specify the directory where your CSV files are located
    directory_path = "drive/MyDrive/accelerometry/datasets"  # Update with the actual directory path

    # Process all files in the directory
    epochs, epochs_per_road = processor.process_all_files(directory_path)

    # Print summary of epochs created
    print("\nEpochs created per road:")
    for road_id, count in epochs_per_road.items():
        print(f"{road_id}: {count} epochs")

    # Optionally, return or further process the epochs for model input
    # Example: epochs could be saved or used in training an anomaly detection model

if __name__ == "__main__":
    main()



Epochs created per road:
Road 4 1: 59 epochs
Road 1a 2: 593 epochs
Road 2 3: 631 epochs
Road 2 2: 688 epochs
Road 2 1: 732 epochs
Road 1b 3: 547 epochs
Road 1a 3: 566 epochs
Road 1b 2: 527 epochs
Road 1b 1: 574 epochs
Road 1 a 1: 597 epochs
Road 5 3: 1212 epochs
Road 5 2: 333 epochs
Road 5 1: 320 epochs
Road 4 2 (1): 49 epochs
Road 4 2: 49 epochs
Road 4c: 49 epochs


In [2]:
# Instantiate the data processor
processor = RoadDataProcessor(epoch_duration=2.0, overlap=0.5, sampling_rate=100)

# Specify the directory containing the CSV files
data_directory = 'drive/MyDrive/accelerometry/datasets'  # Update with your actual directory path

# Load all CSV files from the directory
data_files = processor.load_csv_files(data_directory)

# Process each file in the directory using the same variable names
for road_id, df in data_files.items():
    # Using the same variable names as before (data_path and df)
    data_path = f"{data_directory}/{road_id}.csv"  # Construct the file path for each file

    print(f"Processing {road_id} from {data_path}...")

    # Preprocess the data for the current file
    processed_data = processor.preprocess_data(df)

    # Create epochs for the current file
    epochs = processor.create_epochs(processed_data)

    # Print the number of epochs created for the current file
    print(f"Number of epochs created for {road_id}: {len(epochs)}")

    # Optionally print the first few rows of the first epoch
    print("Sample Epoch:")
    print(epochs[0]['data'].head())



Processing Road 4 1 from drive/MyDrive/accelerometry/datasets/Road 4 1.csv...
Number of epochs created for Road 4 1: 59
Sample Epoch:
           time      ax      ay      az      wx      wy      wz  Latitude  \
0  12:30:17:761  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.000000   
1  12:30:17:762  0.0000  0.0000  0.0000  0.2584  0.0104  0.1405  7.517475   
2  12:30:17:779  0.0000  0.0000  0.0000  0.2584  0.0104  0.1405  7.517475   
3  12:30:17:780  0.7338  0.3279  0.4541  0.2584  0.0104  0.1405  7.517475   
4  12:30:17:781  0.7338  0.3279  0.4541  0.2291 -0.0232  0.1704  7.517475   

   Longitude  Speed (m/s)  Unnamed: 10  acc_magnitude  gyro_magnitude  \
0   0.000000     0.000000          NaN        0.00000        0.000000   
1   4.526368     3.173777          NaN        0.00000        0.294311   
2   4.526368     3.173777          NaN        0.00000        0.294311   
3   4.526368     3.173777          NaN        0.92314        0.294311   
4   4.526368     3.173777          NaN

In [3]:
from typing import List, Dict
import pandas as pd
import numpy as np
from pathlib import Path
from scipy.signal import butter, filtfilt

# Assuming the RoadDataProcessor class is already defined as per your provided code

class RoadDataProcessor:
    def __init__(self, epoch_duration: float = 2.0, overlap: float = 0.5, sampling_rate: int = 100):
        self.epoch_duration = epoch_duration
        self.overlap = overlap
        self.sampling_rate = sampling_rate

    def preprocess_data(self, df: pd.DataFrame) -> pd.DataFrame:
        """Preprocess the data: calculate magnitudes and apply filtering."""
        processed = df.copy()
        processed['acc_magnitude'] = np.sqrt(processed['ax']**2 + processed['ay']**2 + processed['az']**2)
        processed['gyro_magnitude'] = np.sqrt(processed['wx']**2 + processed['wy']**2 + processed['wz']**2)

        # Apply low-pass filter to relevant columns
        def low_pass_filter(data, cutoff=10, fs=100, order=4):
            nyquist = 0.5 * fs
            normal_cutoff = cutoff / nyquist
            b, a = butter(order, normal_cutoff, btype='low', analog=False)
            return filtfilt(b, a, data)

        for col in ['ax', 'ay', 'az', 'acc_magnitude']:
            processed[f'{col}_filtered'] = low_pass_filter(processed[col], cutoff=10, fs=self.sampling_rate, order=4)

        return processed

    def create_epochs(self, df: pd.DataFrame) -> List[Dict]:
        """Segment the data into overlapping epochs."""
        samples_per_epoch = int(self.epoch_duration * self.sampling_rate)
        stride = int(samples_per_epoch * (1 - self.overlap))
        epochs = []

        for start_idx in range(0, len(df) - samples_per_epoch + 1, stride):
            end_idx = start_idx + samples_per_epoch
            epoch_data = df.iloc[start_idx:end_idx]
            epoch = {
                'start_time': epoch_data['time'].iloc[0],
                'end_time': epoch_data['time'].iloc[-1],
                'data': epoch_data
            }
            epochs.append(epoch)

        return epochs


# Assuming data files are loaded from a directory or defined explicitly:
# List of file paths to process (replace with actual paths)
data_paths = [
    'drive/MyDrive/accelerometry/datasets/Road 1 a 1.csv',
    'drive/MyDrive/accelerometry/datasets/Road 1a 2.csv',
    'drive/MyDrive/accelerometry/datasets/Road 1a 3.csv',
    'drive/MyDrive/accelerometry/datasets/Road 1b 1.csv',
    'drive/MyDrive/accelerometry/datasets/Road 1b 2.csv',
    'drive/MyDrive/accelerometry/datasets/Road 1b 3.csv',
    'drive/MyDrive/accelerometry/datasets/Road 2 1.csv',
    'drive/MyDrive/accelerometry/datasets/Road 2 2.csv',
    'drive/MyDrive/accelerometry/datasets/Road 2 3.csv',
    'drive/MyDrive/accelerometry/datasets/Road 4 1.csv',
    'drive/MyDrive/accelerometry/datasets/Road 4 2.csv',
    'drive/MyDrive/accelerometry/datasets/Road 4c.csv',
    'drive/MyDrive/accelerometry/datasets/Road 5 1.csv',
    'drive/MyDrive/accelerometry/datasets/Road 5 2.csv',
    'drive/MyDrive/accelerometry/datasets/Road 5 3.csv',
    # Add more file paths as needed
]

# Initialize the processor
processor = RoadDataProcessor(epoch_duration=2.0, overlap=0.5, sampling_rate=100)

# Initialize a dictionary to store the epochs for each file
all_epochs = {}

# Loop through each file and process it
for data_path in data_paths:
    # Load the CSV file
    df = pd.read_csv(data_path)

    # Preprocess the data
    processed_data = processor.preprocess_data(df)

    # Create epochs from the processed data
    epochs = processor.create_epochs(processed_data)

    # Store the results in the dictionary
    all_epochs[data_path] = epochs

    # Optional: print the number of epochs and a sample from the first epoch
    print(f"Number of epochs created for {data_path}: {len(epochs)}")
    print("Sample Epoch:")
    print(epochs[0]['data'].head())

# At this point, all_epochs will contain the processed epochs for each file


Number of epochs created for drive/MyDrive/accelerometry/datasets/Road 1 a 1.csv: 597
Sample Epoch:
           time      ax      ay      az      wx      wy      wz  Latitude  \
0  12:52:07:885  0.0000  0.0000  0.0000 -0.0470 -0.0055 -0.0232   0.00000   
1  12:52:07:891  0.0000  0.0000  0.0000 -0.0470 -0.0055 -0.0232   7.51795   
2  12:52:07:900  0.0000  0.0000  0.0000 -0.0446  0.0012 -0.0244   7.51795   
3  12:52:07:910  0.0000  0.0000  0.0000 -0.0446  0.0012 -0.0244   7.51795   
4  12:52:07:911  0.0481  0.0029  0.0244 -0.0446  0.0012 -0.0244   7.51795   

   Longitude  Speed (m/s)  Unnamed: 10  acc_magnitude  gyro_magnitude  \
0   0.000000     0.000000          NaN       0.000000        0.052702   
1   4.522062     4.117263          NaN       0.000000        0.052702   
2   4.522062     4.117263          NaN       0.000000        0.050852   
3   4.522062     4.117263          NaN       0.000000        0.050852   
4   4.522062     4.117263          NaN       0.054013        0.050852   

In [4]:
df = df.drop(columns=['Unnamed: 10'], errors='ignore')

In [ ]:
# import matplotlib.pyplot as plt
# plt.figure(figsize=(12, 6))
# plt.plot(processed_data['time'], processed_data['acc_magnitude_filtered'], label='Acc Magnitude (Filtered)', color='blue')
# plt.xlabel('Time')
# plt.ylabel('Filtered Acc Magnitude')
# plt.title('Filtered Acceleration Magnitude Over Time')
# plt.xticks(rotation=45)
# plt.legend()
# plt.grid(True)
# plt.show()

# # Visualization of processed data: gyro magnitude
# plt.figure(figsize=(12, 6))
# plt.plot(processed_data['time'], processed_data['gyro_magnitude'], label='Gyro Magnitude', color='green')
# plt.xlabel('Time')
# plt.ylabel('Gyro Magnitude')
# plt.title('Gyro Magnitude Over Time')
# plt.xticks(rotation=45)
# plt.legend()
# plt.grid(True)
# plt.show()


NameError: name 'processed_data' is not defined

<Figure size 1200x600 with 0 Axes>

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# Initialize an empty list to store the processed feature matrices
all_features = []

# Loop through each dataset (file) in the all_epochs dictionary
for data_path, epochs in all_epochs.items():
    # Extract features from each epoch
    for epoch in epochs:
        # Assuming `processed_data` is the 'data' of each epoch
        epoch_data = epoch['data']

        # Prepare the feature matrix for each epoch
        features = epoch_data[['ax_filtered', 'ay_filtered', 'az_filtered', 'acc_magnitude_filtered']]
        features = features.to_numpy()  # Convert to NumPy array

        # Append features to the all_features list
        all_features.append(features)

# Convert the list of feature arrays into a single NumPy array
all_features = np.vstack(all_features)

# Step 1: Train-Test Split
X_train, X_test = train_test_split(all_features, test_size=0.2, random_state=42, shuffle=True)

# Step 2: Normalize or Standardize the Data
scaler = StandardScaler()  # Use MinMaxScaler() for normalization
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Output the shapes of the train and test sets
X_train_scaled.shape, X_test_scaled.shape



((1196320, 4), (299080, 4))

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Assuming `all_features` is the combined features from all datasets

# Step 1: Prepare the dataset (as previously done)
# all_features = np.vstack(all_features)  # Combined features from multiple datasets

X_train_scaled, X_test_scaled = train_test_split(all_features, test_size=0.2, random_state=42, shuffle=True)

# Step 2: Design the Autoencoder
def build_autoencoder_learning_rate(input_dim):
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(32, activation='relu')(input_layer)
    encoded = Dense(16, activation='relu')(encoded)
    encoded = Dense(8, activation='relu')(encoded)  # Latent representation

    decoded = Dense(16, activation='relu')(encoded)
    decoded = Dense(32, activation='relu')(decoded)
    output_layer = Dense(input_dim, activation='sigmoid')(decoded)

    autoencoder = Model(inputs=input_layer, outputs=output_layer)

    # Using the Adam optimizer with learning rate of 0.001
    autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return autoencoder

# Build the autoencoder model
autoencoder = build_autoencoder_learning_rate(X_train_scaled.shape[1])

# Step 3: Train the Autoencoder
history = autoencoder.fit(
    X_train_scaled, X_train_scaled,  # Train to reconstruct itself
    epochs=38,  # Increase epochs to give the model more time to learn
    batch_size=32,
    shuffle=True,
    validation_split=0.2,  # Use 20% of training data for validation
    verbose=1
)

# Step 4: Evaluate on Test Data
# Reconstruct the test set
X_test_reconstructed = autoencoder.predict(X_test_scaled)

# Calculate the reconstruction error for each test sample
reconstruction_errors = np.mean(np.square(X_test_scaled - X_test_reconstructed), axis=1)

# Step 5: Set a Threshold for Anomalies
# Use a simple approach like 95th percentile of reconstruction errors
threshold = np.percentile(reconstruction_errors, 95)

# Flag anomalies based on the threshold
anomalies = reconstruction_errors > threshold

# Step 6: Output Results
print(f"Threshold for anomalies: {threshold}")
print(f"Number of anomalies detected: {np.sum(anomalies)}")

# Visualize reconstruction errors
plt.hist(reconstruction_errors, bins=50, color='blue', alpha=0.7)
plt.axvline(x=threshold, color='red', linestyle='--', label='Threshold')
plt.title('Reconstruction Errors')
plt.xlabel('Error')
plt.ylabel('Frequency')
plt.legend()
plt.show()


Epoch 1/38
29908/29908 ━━━━━━━━━━━━━━━━━━━━ 64s 2ms/step - loss: 0.1309 - val_loss: 0.1226
Epoch 2/38
29908/29908 ━━━━━━━━━━━━━━━━━━━━ 78s 2ms/step - loss: 0.1213 - val_loss: 0.1224
Epoch 3/38
29908/29908 ━━━━━━━━━━━━━━━━━━━━ 58s 2ms/step - loss: 0.1218 - val_loss: 0.1224
Epoch 4/38
29908/29908 ━━━━━━━━━━━━━━━━━━━━ 54s 2ms/step - loss: 0.1235 - val_loss: 0.1224
Epoch 5/38
29908/29908 ━━━━━━━━━━━━━━━━━━━━ 60s 2ms/step - loss: 0.1223 - val_loss: 0.1224
Epoch 6/38
